# Homework 3

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from collections import namedtuple
import itertools
import numpy as np


# Take a moment to review the data structure below.
class FrozenMap:
  """A dict-like data structure that is immutable and hashable.

  The keys and values must be hashable and immutable too.

  Example usage:
    fm = FrozenMap(a="hello", b="world")
    
    for k in fm:
      print(fm[k])

    for k in fm.keys():
      print(fm[k])
    
    for v in fm.values():
      print(v)
    
    for k, v in fm.items():
      print(k, v)
    
    fm2 = FrozenMap(c=fm)
    print(fm2["c"]["a"])  # "hello"
    print(fm2["c"]["b"])  # "world"
  """

  def __init__(self, *args, **kwargs):
    self._dict = dict(*args, **kwargs)
    self._as_frozenset = frozenset(self._dict.items())
    self._hash = hash(self._as_frozenset)

  def __getitem__(self, key):
    return self._dict[key]

  def __iter__(self):
    return iter(self._dict)

  def __hash__(self):
    return self._hash

  def __eq__(self, other):
    return hash(self) == hash(other)

  def __str__(self):
    return str(self._dict)

  def __repr__(self):
    return f'FrozenMap({repr(self._dict)})'

  def keys(self):
    return self._dict.keys()

  def values(self):
    return self._dict.values()

  def items(self):
    return self._dict.items()


# A random variable; name is str, dim is int
RV = namedtuple("RV", ["name", "dim"])
# rvs is a frozenset of RVs; table is a FrozenMap
# of the form {{RV: int}: float}, where the keys are
# rv joint assignments that are also FrozenMaps.
Potential = namedtuple("Potential", ["rvs", "table"])

InferenceProblem = namedtuple("InferenceProblem",
  ["rvs", "potentials", "query", "evidence"])


def iter_joint_assignments(rvs):
  """Iterates over joint assignments for a list of RVs.

  Args:
    rvs: A list of RVs.

  Yields:
    assignment: A FrozenMap.
  """
  domains = [range(rv.dim) for rv in rvs]
  for assignment_values in itertools.product(*domains):
    yield FrozenMap(zip(rvs, assignment_values))


def create_potential_from_array(rvs, array):
  """Create a potential from a list of RVs and a numpy array.

  The order of the random variables corresponds to the axes
  of the numpy array.

  This is a helper for creating problems; it may not be
  necessary for your solutions.

  Args:
    rvs: A list of RVs.
    array: A numpy array of potential values.

  Returns:
    potential: A FrozenMap from assignments (FrozenMaps) to values.
  """
  assert len(rvs) == len(array.shape)
  assert all(rv.dim == dim for (rv, dim) in zip(rvs, array.shape))
  table = {}
  domains = [range(rv.dim) for rv in rvs]
  for assignment_values in itertools.product(*domains):
    assignment = FrozenMap(zip(rvs, assignment_values))
    table[assignment] = array[assignment_values]
  table = FrozenMap(table)
  potential = Potential(frozenset(rvs), table)
  return potential

# Problems for unit tests

def create_debug_2vars_problem(version):
  """A simple problem with two random variables"""
  A = RV("A", 2)
  B = RV("B", 3)
  rvs = [A, B]

  p_a_given_b = Potential(
    rvs=frozenset({A, B}),
    table=FrozenMap({
      FrozenMap({A: 0, B: 0}): 0.9,
      FrozenMap({A: 1, B: 0}): 0.1,
      FrozenMap({A: 0, B: 1}): 0.15,
      FrozenMap({A: 1, B: 1}): 0.85,
      FrozenMap({A: 0, B: 2}): 0.44,
      FrozenMap({A: 1, B: 2}): 0.56,
    })
  )

  p_b = Potential(
    rvs=frozenset({B}),
    table=FrozenMap({
      FrozenMap({B: 0}): 0.7,
      FrozenMap({B: 1}): 0.2,
      FrozenMap({B: 2}): 0.1,
    })
  )

  pots = [p_a_given_b, p_b]
  if version == 1:
    query = {A : 1}
    evidence = {B : 1}
  elif version == 2:
    query = {B : 1}
    evidence = {A : 1}        
  else:
    assert version == 3
    query = {A : 1, B : 1}
    evidence = {}
  
  return InferenceProblem(rvs, pots, query, evidence)


def create_california_problem(version):
  """Holmes, watson, earthquakes, radios, oh my...
  """
  p_b = np.array([0.99, 0.01])
  p_e = np.array([0.97, 0.03])
  p_re = np.array([
      [0.98, 0.01],
      [0.02, 0.99],
  ])
  p_aeb = np.zeros((2, 2, 2))
  p_aeb[1, 0, 0] = 0.01
  p_aeb[0, 0, 0] = 1. - 0.01
  p_aeb[1, 0, 1] = 0.2
  p_aeb[0, 0, 1] = 1. - 0.2
  p_aeb[1, 1, 0] = 0.95
  p_aeb[0, 1, 0] = 1. - 0.95
  p_aeb[1, 1, 1] = 0.96
  p_aeb[0, 1, 1] = 1. - 0.96

  A = RV("Alarm", 2)
  B = RV("Burglar", 2)
  E = RV("Earthquake", 2)
  R = RV("Radio", 2)
  rvs = [A, B, E, R]
  pots = [
      create_potential_from_array([B], p_b),
      create_potential_from_array([E], p_e),
      create_potential_from_array([R, E], p_re),
      create_potential_from_array([A, E, B], p_aeb)
  ]
  if version == "alarm":
      # P(B=1 | A=1)
      query = {B : 1}
      evidence = {A : 1}
  else:
      assert version == "alarm and earthquake"
      # P(B=1 | A=1, R=1)
      query = {B : 1}
      evidence = {A : 1, R : 1}
  return InferenceProblem(rvs, pots, query, evidence)


def create_binary_chain_problem(num_vars):
  """A simple binary chain designed to stress test inference
  """
  rvs = [RV(f"X{i}", 2) for i in range(num_vars)]
  pots = []
  for rv_t, rv_t1 in zip(rvs[:-1], rvs[1:]):
      pot = create_potential_from_array(
        [rv_t, rv_t1], np.array([
          [0.9, 0.1],
          [0.1, 0.9],
      ]))
      pots.append(pot)
  query = {rvs[0] : 0}
  evidence = {}        
  return InferenceProblem(rvs, pots, query, evidence)


def query_potential_from_rv_name_dict(rv_name_dict, potential):
  """Given a dict from RV names (strs) to assignments,
  return the corresponding value in the potential table.

  Args:
    rv_name_dict: A dict from str names to values.
    potential: A Potential.

  Returns:
    value: The float value from potential.table.
  """
  name_to_rv = {rv.name : rv for rv in potential.rvs}
  assert all(name in name_to_rv for name in rv_name_dict)
  assignment = FrozenMap({name_to_rv[n]: v for (n, v) in rv_name_dict.items()})
  return potential.table[assignment]


def get_sub_assignment(fm, keys):
  """Given a FrozenMap and a subset of keys, return a new FrozenMap.

  Args:
    fm: A FrozenMap.
    keys: A subset of the keys in fm.

  Returns:
    new_fm: A FrozenMap.
  """
  return FrozenMap({k: fm[k] for k in keys})


def postprocess_potential(potential, decimals=6):
  """Helper for potentials_equal.

  Rounds to avoid numerical issues.
  """
  return frozenset((assignment, np.round(p, decimals=decimals))
                   for assignment, p in potential.table.items())


def potentials_equal(potential1, potential2, decimals=6):
  """Check whether two potentials are (nearly) equal.
  """
  if potential1.rvs != potential2.rvs:
    return False
  res1 = postprocess_potential(potential1, decimals=decimals)
  res2 = postprocess_potential(potential2, decimals=decimals)
  return res1 == res2


def run_belief_prop(problem, max_iters=100):
  """Run inference with belief propagation.
  
  Calls `run_single_marginal_bp`, which is for
  you to implement.
  
  Args:
    problem: InferenceProblem
    max_iters: int
       Maximum number of iterations for each BP call.
  Returns:
    result: float
      Answer to the query in the problem
  """
  # Convert evidence to potentials
  problem = convert_evidence_to_potentials(problem)
  # Convert joint queries into a sequence of marginal queries
  problems = []
  # Set up arbitrary order for chain rule
  queries = sorted(problem.query.items())
  for i, (rv, val) in enumerate(queries):
    # P(rv = val | previous vals)
    new_query = {rv : val}
    new_evidence = dict(queries[:i])
    new_problem = InferenceProblem(problem.rvs,
                                   problem.potentials,
                                   new_query, new_evidence)
    new_problem = convert_evidence_to_potentials(new_problem)
    problems.append(new_problem)
  # Run individual problems
  result = 1.
  for p in problems:
    (query_rv, query_val), = p.query.items()
    p_result = run_single_marginal_bp(query_rv, query_val,
      p.rvs, p.potentials, max_iters=max_iters)
    result *= p_result
  return result


def convert_evidence_to_potentials(problem):
  """Create singleton potentials to account for evidence.
  Helper for run_belief_prop.
  """
  new_problem = InferenceProblem(problem.rvs,
                                 [pot for pot in problem.potentials],
                                 problem.query.copy(), {})
  for rv, val in problem.evidence.items():
    # Create singleton potential
    table = np.zeros(rv.dim)
    table[val] = 1.
    new_potential = create_potential_from_array([rv], table)
    new_problem.potentials.append(new_potential)
  return new_problem



## Problems

### Warming Up to Potentials
Write a function that returns a potential following the description in the docstring below. There are two ways to write this function, and we give examples in the colab notebook. One way is shown in the example function `create_debug_2vars_problem`, where the potential is created by listing each combination of the random variable values and the associated probability. The second way is shown in the function `create_california_problem` that uses the helper function `create_potential_from_array`.

For reference, our solution is **13** lines of code.

In [ ]:
def warmup():
  """Creates a potential involving two RVs.

  * The RVs should be called "Rain" and "Clouds".
  * They should both be dimension 2.
  * The potential table should have the following values:
  *   Rain=0, Clouds=0 : 0.6
  *   Rain=1, Clouds=0 : 0.05
  *   Rain=0, Clouds=1 : 0.15
  *   Rain=1, Clouds=1 : 0.2

  We are expecting your method to return a Potential. This is a class
  we have provided for you, and if you examine the colab notebook, you will
  see some documentation and some helper functions.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
my_potential = warmup()
assert isinstance(my_potential.rvs, frozenset)
assert len(my_potential.rvs) == 2
rain_rv, cloud_rv = None, None
for rv in my_potential.rvs:
  assert rv.dim == 2
  if rv.name == 'Rain':
    rain_rv = rv
  elif rv.name == 'Clouds':
    cloud_rv = rv
  else:
    assert False, "Unexpected RV name"
assert isinstance(my_potential.table, FrozenMap)
for k, v in my_potential.table.items():
  assert isinstance(k, FrozenMap)
  assert frozenset(k.keys()) == my_potential.rvs
  if k == FrozenMap({rain_rv: 0, cloud_rv: 0}):
    assert v == 0.6
  elif k == FrozenMap({rain_rv: 1, cloud_rv: 0}):
    assert v == 0.05
  elif k == FrozenMap({rain_rv: 0, cloud_rv: 1}):
    assert v == 0.15
  elif k == FrozenMap({rain_rv: 1, cloud_rv: 1}):
    assert v == 0.2
  else:
    assert False, "Unexpected potential table key"

print('Tests passed.')

### Warm Up Part 2
Write a function that queries the given potential for the specific variable values described in the docstring. You may find the helper function `query_potential_from_rv_name_dict` useful.

For reference, our solution is **1** lines of code.

In [ ]:
def warmup2(ab_potential):
  """Given a potential involving RVs 'A' and 'B',
  return the potential value for A = 0, B = 1.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

assert warmup2(Potential(frozenset({RV(name="A", dim=2), RV(name="B", dim=2)}), {FrozenMap({RV(name="A", dim=2): 0, RV(name="B", dim=2): 0}): 0.98, FrozenMap({RV(name="A", dim=2): 0, RV(name="B", dim=2): 1}): 0.01, FrozenMap({RV(name="A", dim=2): 1, RV(name="B", dim=2): 0}): 0.02, FrozenMap({RV(name="A", dim=2): 1, RV(name="B", dim=2): 1}): 0.99})) == 0.01
print('Tests passed.')

### Potential Multiplication
Write a function that multiplies a list of potentials together. (Make sure to refer to the top of the colab notebook, especially the functions `iter_joint_assignments` and `get_sub_assignment`.)

For reference, our solution is **25** lines of code.

In [ ]:
def multiply_potentials(potentials):
  """Multiply potentials together.

  Args:
    potentials: A list of Potentials.

  Returns:
    result: A new Potential.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
A, B = RV('A', 3), RV('B', 2)
# A matches B
pot1 = create_potential_from_array(rvs=[A, B],
  array=np.array([
  [1, 0],
  [0, 1],
  [0, 0],
]))
# B is 0
pot2 = create_potential_from_array(rvs=[B],
  array=np.array([1, 0]))
# A and B are both 0
expected_result_table = np.zeros((3, 2))
expected_result_table[0, 0] = 1
expected_result = create_potential_from_array(rvs=[A, B], array=expected_result_table)
result = multiply_potentials([pot1, pot2])
assert potentials_equal(result, expected_result)

A, B, C = RV('A', 2), RV('B', 2), RV('C', 2)
# A is definitely 0, B could be either
pot1 = create_potential_from_array(rvs=[A, B],
  array=np.array([
  [1, 1],
  [0, 0]
]))
# B is definitely 0 and C could be either
pot2 = create_potential_from_array(rvs=[B, C],
  array=np.array([
  [1, 1],
  [0, 0]
]))
# A is 0, B is 0, and C is either
expected_result_table = np.zeros((2, 2, 2))
expected_result_table[0, 0, :] = 1
expected_result = create_potential_from_array(
  rvs=[A, B, C],
  array=expected_result_table)
result = multiply_potentials([pot1, pot2])
assert potentials_equal(result, expected_result)

print('Tests passed.')

### Marginalization
Write a function that marginalizes out given variables of a potential to create a new potential. You most likely find the helper functions such as `get_sub_assignment` useful.

For reference, our solution is **10** lines of code.

In [ ]:
def marginalize(potential, rvs):
  """Create a new potential where each rv has been marginalized out.

  Args:
    potential: A Potential.
    rvs: A set of random variables in the potential.

  Returns:
    new_potential: A Potential.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
A, B = RV('A', 2), RV('B', 3)
pot = create_potential_from_array(rvs=[A, B],
  array=np.array([
    [0.54, 0.16, 0.01],
    [0.04, 0.06, 0.19],
  ]))
expected_result = create_potential_from_array(rvs=[B],
  array=np.array([0.58, 0.22, 0.20]))
result = marginalize(pot, {A})
assert potentials_equal(result, expected_result)

print('Tests passed.')

### Neighbor Relations
Given a potential and a variable attached to the potential, returns a set of the other variables attached to that potential. (Make sure your method returns a set and not a list or tuple.)

For reference, our solution is **1** lines of code.

In [ ]:
def neighbors(P, V):
  """Given a potential P and a random variable V, 
  return the set of all the other neighbours of P that aren't V. 

  Args:
    P: A Potential.
    V: A RV.

  Returns:
    other_rvs: A set of RVs that aren't V.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
A, B, C = RV('A', 2), RV('B', 2), RV('C', 2)
pot = create_potential_from_array(rvs=[A, B, C],
  array=np.ones((2, 2, 2)))
assert neighbors(pot, A) == {B, C}

print('Tests passed.')

### Sum Product
Write a function that runs sum product to determine the marginal probability of a single random variable assignment.

For reference, our solution is **55** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `marginalize`, `multiply_potentials`, `neighbors`. You may not need to use all of them.

In [ ]:
def run_single_marginal_bp(query_rv, query_val, rvs, potentials, max_iters=100):
  """Run belief propagation on a problem with a single query
  and no evidence.

  Should terminate early if old and new messages are close
  enough, as defined by potentials_equal.

  Args:
    query_rv: A RV.
    query_val: A value in the domain of query_rv.
    rvs: All the RVs in the problem.
    potentials: All the Potentials in the problem.
    max_iters: Maximum number of iterations to run
      belief propagation.

  Returns:
    marginal_val: the float probability of query_rv = query_val.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# The binary chain problem involves a single marginal
# query with no evidence
problem = create_binary_chain_problem(3)
(query_rv, query_val), = problem.query.items()
assert len(problem.evidence) == 0
result = run_single_marginal_bp(query_rv, query_val, problem.rvs, problem.potentials)
assert abs(result - 0.5) < 1e-5

# Run full belief propagation, which calls
# `run_single_marginal_bp` as a subroutine
result = run_belief_prop(create_debug_2vars_problem(1))
assert abs(result - 0.85) < 1e-5
result = run_belief_prop(create_debug_2vars_problem(2))
assert abs(result - 0.574324) < 1e-5
result = run_belief_prop(create_debug_2vars_problem(3))
assert abs(result - 0.17) < 1e-5
result = run_belief_prop(create_binary_chain_problem(5))
assert abs(result - 0.5) < 1e-5
result = run_belief_prop(create_binary_chain_problem(25))
assert abs(result - 0.5) < 1e-5
result = run_belief_prop(create_california_problem("alarm"))
assert abs(result - 0.055636) < 1e-5
result = run_belief_prop(create_california_problem("alarm and earthquake"))
assert abs(result - 0.011386) < 1e-5
print('Tests passed.')